In [15]:
import pandas as pd
import talib as ta
import matplotlib.pyplot as plt
import warnings
import lightgbm as lgb
from workflow import main_indicators,Larry_completo_teste,Larry_completo
from sklearn.model_selection import train_test_split
import plotly.express as px
from datetime import datetime
warnings.filterwarnings("ignore")

In [109]:
ativo = 'wdo_1_full'
colnames = ['Data','Open','High','Low','Close','Vol']
df = pd.read_csv(f'./Dados/{ativo}.csv', names = colnames, skiprows=1)

#df = df.iloc[:1000]

In [17]:
ativo = 'EURUSD5'
colnames = ['Data','Open','High','Low','Close','Vol']
df = pd.read_csv(f'./Dados/{ativo}.csv', sep = '\t', names = colnames)
#df = df[df['Data'] > '2022-01-01']

In [18]:
df = main_indicators(df)

df_acao = Larry_completo(df, tempo_fechar=-5, excluir_seguidos = True, acao = 'all', x_stop = 5)

df_acao['Hora'] = df_acao['Data'].apply(lambda x: x[11:])
df_acao['Hora_h'] = df_acao['Hora'].apply(lambda x: x[:2])

#df_acao = df_acao[df_acao['tend'] == 0]

compra = df_acao[df_acao['acao'] =='call']['resultado_binario'].sum()/df_acao[df_acao['acao'] =='call'].shape[0]
venda = df_acao[df_acao['acao'] =='sell']['resultado_binario'].sum()/df_acao[df_acao['acao'] =='sell'].shape[0]

compra_result = df_acao[df_acao['acao'] =='call']['resultado_valor'].sum()
venda_result = df_acao[df_acao['acao'] =='sell']['resultado_valor'].sum()

print(f'Venda: {venda}% -- Compra: {compra}%')
print(f'Venda: {venda_result} -- Compra: {compra_result}')

df_acao['Hora'] = df_acao['Data'].apply(lambda x: x[11:])
df_acao['Hora_h'] = df_acao['Hora'].apply(lambda x: x[:2])
df_acao['Mes'] = df_acao['Data'].apply(lambda x: x[5:7])

#Retirar hora especifica
#df_acao = df_acao[df_acao['Hora_h'] != '10']


df_new_hora = df_acao[['Hora_h','resultado_binario','resultado_valor']]
df_new_mes = df_acao[['Mes','resultado_binario','resultado_valor']]

print('Agrupado por Mes')

print(df_new_mes.groupby(['Mes']).mean())

print(df_new_mes.groupby(['Mes']).sum())

print('')
print('---------------------------------------------------')

Venda: 0.5201393728222996% -- Compra: 0.5223165388828039%
Venda: 0.15046000000000548 -- Compra: 0.1284599999999858
Agrupado por Mes
     resultado_binario  resultado_valor
Mes                                    
01            0.505643         0.000002
02            0.525872         0.000043
03            0.528932         0.000058
04            0.509091         0.000011
05            0.516962         0.000022
06            0.520058         0.000019
07            0.514607         0.000016
08            0.551675         0.000053
09            0.523948         0.000016
10            0.534328         0.000016
11            0.510240        -0.000012
12            0.500550        -0.000026
     resultado_binario  resultado_valor
Mes                                    
01                 448          0.00146
02                 437          0.03573
03                 649          0.07151
04                 672          0.01397
05                 701          0.03026
06                 713      

In [34]:
df = df_acao.copy()
#df = df[df['acao'] == 'sell']
df['Hora_h'] = df['Hora_h'].apply(lambda x: int(x))

df = df.drop(columns = ['MA_9_diff_value', 'MA_9_tend', 'MA_9_tend_diff', 'preco','stop', 'realizacao_simplista', 'stop_real', 'tend','Hora','Mes'])

inicio = '2022-01-01'
termino = '2022-01-01'

df_train = df[(df['Data'] > '2020-01-01') & (df_acao['Data'] < inicio)]
df_train = df_train.set_index(['Data','resultado_valor','acao'])

df_test = df[(df['Data'] > inicio)]
df_test = df_test.set_index(['Data','resultado_valor','acao'])

y_train = df_train['resultado_binario']
x_train = df_train.drop(columns=['resultado_binario'])

y_test = df_test['resultado_binario']
x_test = df_test.drop(columns=['resultado_binario'])

In [35]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import lightgbm as lgb

In [42]:
clf = ExtraTreesClassifier()
#clf = LogisticRegression()
#clf = lgb.LGBMClassifier()
clf.fit(x_train, y_train)

# predict the results
y_pred=clf.predict(x_test)
y_proba = clf.predict_proba(x_test)[:,1]
# view accuracy
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))

print('Training set score: {:.4f}'.format(clf.score(x_train, y_train)))

print('Test set score: {:.4f}'.format(clf.score(x_test, y_test)))

LightGBM Model accuracy score: 0.5136
Training set score: 1.0000
Test set score: 0.5136


In [43]:
df_thr = x_test.copy()
df_thr = df_thr.reset_index()
df_thr['pred'] = y_pred
df_thr['proba'] = y_proba
df_thr['resultado_binario'] = y_test.values

lista_full = []
for thr in [0.5,0.6,0.7,0.8]:

    df_temp = df_thr[df_thr['proba'] > thr]

    qtd_operacoes = df_temp.shape[0]

    acc = df_temp[df_temp['resultado_binario'] ==1].shape[0]/qtd_operacoes

    valor = df_temp['resultado_valor'].sum()

    lista_full.append([thr,qtd_operacoes,acc,valor])

df_a = pd.DataFrame(lista_full)
df_a.columns = ['thr','qtd_operacoes','acc','valor']
df_a

,thr,qtd_operacoes,acc,valor
0,0.5,2656,0.526355,0.07655
1,0.6,600,0.508333,0.01974
2,0.7,33,0.545455,-0.00056
3,0.8,1,1.000000,0.00100


In [14]:
thr = 0.6
novo = df_thr[df_thr['proba'] > thr]

compra_result = novo[novo['acao'] =='call']['resultado_valor'].sum()
venda_result = novo[novo['acao'] =='sell']['resultado_valor'].sum()

print(f'Venda: {venda}% -- Compra: {compra}%')
print(f'Venda: {venda_result} -- Compra: {compra_result}')

novo['Hora'] = novo['Data'].apply(lambda x: x[11:])
novo['Hora_h'] = novo['Hora'].apply(lambda x: x[:2])
novo['Mes'] = novo['Data'].apply(lambda x: x[5:7])

#Retirar hora especifica
#df_acao = df_acao[df_acao['Hora_h'] != '10']


df_new_hora = novo[['Hora_h','resultado_binario','resultado_valor']]
df_new_mes = novo[['Mes','resultado_binario','resultado_valor']]

print('Agrupado por Mes')

print(df_new_mes.groupby(['Mes']).mean())

print(df_new_mes.groupby(['Mes']).sum())

print('')
print('---------------------------------------------------')

Venda: 0.5154921853578284% -- Compra: 0.5169991570665917%
Venda: 0.02453999999999934 -- Compra: 0.03254000000000001
Agrupado por Mes
     resultado_binario  resultado_valor
Mes                                    
01            0.623529         0.000178
02            0.475410         0.000072
03            0.586207         0.000084
04            0.506329         0.000046
05            0.395349         0.000122
06            0.442857        -0.000127
07            0.750000        -0.000085
08            0.551724         0.000177
09            0.588235         0.000168
10            0.666667         0.000387
11            0.250000        -0.000313
12            0.642857         0.000549
     resultado_binario  resultado_valor
Mes                                    
01                  53          0.01509
02                  29          0.00439
03                  51          0.00732
04                  40          0.00364
05                  17          0.00526
06                  31     

In [21]:
clf.feature_importances_

array([0.02798949, 0.02760532, 0.02806069, 0.02815617, 0.04163741,
       0.02886301, 0.02809464, 0.01860846, 0.0287792 , 0.02808466,
       0.02842642, 0.03630936, 0.03642234, 0.03614883, 0.03585711,
       0.03916325, 0.03858573, 0.03519122, 0.03777918, 0.03755228,
       0.03724774, 0.03817417, 0.03899376, 0.03541305, 0.03684869,
       0.03769752, 0.03493416, 0.03487815, 0.02848586, 0.03001213])

In [22]:
clf.feature_names_in_

array(['Open', 'High', 'Low', 'Close', 'Vol', 'MA_5', 'MA_21', 'SAR',
       'BB_up', 'BB_mid', 'BB_low', 'OBV', 'macd', 'macdsignal',
       'macdhist', 'fastk', 'fastd', 'RSI', 'MOM', 'TRIX', 'CCI', 'ADX',
       'dist_bb', 'dist_media', 'dist_media_abs', 'dist_price_media',
       'dist_RSI_70', 'dist_RSI_30', 'MA_9', 'MA_100'], dtype=object)